In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    #print("nse3 : ", nse3)
    #print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'geum_auto_13'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2013.xlsx', '현도_2014.xlsx', '현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2013.xlsx', '대청호_2014.xlsx', '대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx'], ['장계_2013.xlsx', '장계_2014.xlsx', '장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx', '장계_2019.xlsx', '장계_2020.xlsx']]
data/geum/자동/현도_2013.xlsx
data/geum/자동/현도_2014.xlsx
data/geum/자동/현도_2015.xlsx
data/geum/자동/현도_2016.xlsx
data/geum/자동/현도_2017.xlsx
data/geum/자동/현도_2018.xlsx
data/geum/자동/현도_2019.xlsx
data/geum/자동/현도_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-10-31 23:00
data/geum/자동/대청호_2013.xlsx
data/geum/자동/대청호_2014.xlsx
data/geum/자동/대청호_2015.xlsx
data/geum/자동/대청호_2016.xlsx
data/geum/자동/대청호_2017.xlsx
data/geum/자동/대청호_2018.xlsx
data/geum/자동/대청호_2019.xlsx
data/geum/자동/대청호_2020.xlsx
time range in fi

Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.2864 - disc_loss: 0.3489 - rmse: 0.3393 - val_loss: 0.5689
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.9237 - disc_loss: 0.3411 - rmse: 0.4466 - val_loss: 0.6134
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.0388 - disc_loss: 0.3793 - rmse: 0.3313 - val_loss: 0.5658
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.8051 - disc_loss: 0.3626 - rmse: 0.3405 - val_loss: 0.3434
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7478 - disc_loss: 0.3495 - rmse: 0.3298 - val_loss: 0.3527
Epoch 55/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.6437 - disc_loss: 0.3472 - rmse: 0.3684 - val_loss: 0.3011
Epoch 56/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.9355 - disc_loss: 0.3481 - rmse: 0.3176 - val_loss: 0.3240
Epoch 57/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4779 - disc_loss: 0.2880 - rmse: 0.2353 - val_loss: 0.2595
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0366 - disc_loss: 0.2806 - rmse: 0.2824 - val_loss: 0.2743
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8197 - disc_loss: 0.2809 - rmse: 0.2432 - val_loss: 0.3398
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4256 - disc_loss: 0.2939 - rmse: 0.2758 - val_loss: 0.3169
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4664 - disc_loss: 0.3089 - rmse: 0.2666 - val_loss: 0.3102
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4055 - disc_loss: 0.2803 - rmse: 0.2495 - val_loss: 0.2599
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7479 - disc_loss: 0.2812 - rmse: 0.2451 - val_loss: 0.2614
Epoch 115/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7524 - disc_loss: 0.2668 - rmse: 0.2598 - val_loss: 0.2466
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4685 - disc_loss: 0.2644 - rmse: 0.3163 - val_loss: 0.3367
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1624 - disc_loss: 0.2593 - rmse: 0.2454 - val_loss: 0.2371
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9764 - disc_loss: 0.2563 - rmse: 0.3253 - val_loss: 0.2297
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8010 - disc_loss: 0.2726 - rmse: 0.2643 - val_loss: 0.2583
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4926 - disc_loss: 0.2640 - rmse: 0.2369 - val_loss: 0.2622
Epoch 172/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1283 - disc_loss: 0.2518 - rmse: 0.2203 - val_loss: 0.2269
Epoch 173/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3398 - disc_loss: 0.2345 - rmse: 0.2195 - val_loss: 0.2510
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3427 - disc_loss: 0.2365 - rmse: 0.2352 - val_loss: 0.3015
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2997 - disc_loss: 0.2198 - rmse: 0.2727 - val_loss: 0.2574
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0266 - disc_loss: 0.2425 - rmse: 0.2615 - val_loss: 0.2648
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9457 - disc_loss: 0.2424 - rmse: 0.2645 - val_loss: 0.2543
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3441 - disc_loss: 0.2439 - rmse: 0.3291 - val_loss: 0.2418
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4738 - disc_loss: 0.2244 - rmse: 0.2469 - val_loss: 0.2403
Epoch 231/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3542 - disc_loss: 0.1713 - rmse: 0.1891 - val_loss: 0.2398
Epoch 1153/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1545 - disc_loss: 0.1642 - rmse: 0.2025 - val_loss: 0.2113
Epoch 1154/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8703 - disc_loss: 0.1728 - rmse: 0.2552 - val_loss: 0.2277
Epoch 1155/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4196 - disc_loss: 0.1726 - rmse: 0.2312 - val_loss: 0.3564
Epoch 1156/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.1970 - disc_loss: 0.1782 - rmse: 0.1668 - val_loss: 0.2383
Epoch 1157/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7824 - disc_loss: 0.1732 - rmse: 0.2214 - val_loss: 0.2162
Epoch 1158/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 16.5026 - disc_loss: 0.1721 - rmse: 0.6837 - val_loss: 0.2109
Epoch 1159/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3814 - disc_loss: 0.1697 - rmse: 0.1978 - val_loss: 0.1692
Epoch 1211/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9365 - disc_loss: 0.1721 - rmse: 0.2419 - val_loss: 0.2165
Epoch 1212/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4624 - disc_loss: 0.1726 - rmse: 0.2466 - val_loss: 0.2303
Epoch 1213/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.2999 - disc_loss: 0.1763 - rmse: 0.2349 - val_loss: 0.1822
Epoch 1214/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3359 - disc_loss: 0.1726 - rmse: 0.1796 - val_loss: 0.2001
Epoch 1215/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3322 - disc_loss: 0.1694 - rmse: 0.1863 - val_loss: 0.1904
Epoch 1216/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0079 - disc_loss: 0.1702 - rmse: 0.1912 - val_loss: 0.2362
Epoch 1217/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9580 - disc_loss: 0.1709 - rmse: 0.2101 - val_loss: 0.2534
Epoch 1269/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.4580 - disc_loss: 0.1695 - rmse: 0.2122 - val_loss: 0.2235
Epoch 1270/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 8.3739 - disc_loss: 0.1744 - rmse: 0.5290 - val_loss: 0.2278
Epoch 1271/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.5074 - disc_loss: 0.1664 - rmse: 0.1682 - val_loss: 0.1919
Epoch 1272/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.3888 - disc_loss: 0.1687 - rmse: 0.2213 - val_loss: 0.1816
Epoch 1273/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.6102 - disc_loss: 0.1602 - rmse: 0.1829 - val_loss: 0.2808
Epoch 1274/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.0552 - disc_loss: 0.1735 - rmse: 0.2058 - val_loss: 0.2022
Epoch 1275/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4324 - disc_loss: 0.1630 - rmse: 0.1915 - val_loss: 0.1908
Epoch 1327/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4467 - disc_loss: 0.1514 - rmse: 0.1827 - val_loss: 0.1790
Epoch 1328/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8893 - disc_loss: 0.1567 - rmse: 0.1841 - val_loss: 0.2017
Epoch 1329/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4304 - disc_loss: 0.1587 - rmse: 0.1714 - val_loss: 0.2270
Epoch 1330/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6931 - disc_loss: 0.1650 - rmse: 0.1762 - val_loss: 0.2666
Epoch 1331/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2612 - disc_loss: 0.1637 - rmse: 0.1708 - val_loss: 0.2010
Epoch 1332/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 6.0932 - disc_loss: 0.1771 - rmse: 0.2966 - val_loss: 0.2408
Epoch 1333/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 2.5046 - disc_loss: 0.1725 - rmse: 0.1983 - val_loss: 0.1795
Epoch 1385/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.3145 - disc_loss: 0.1615 - rmse: 0.2219 - val_loss: 0.1810
Epoch 1386/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5478 - disc_loss: 0.1743 - rmse: 0.2012 - val_loss: 0.1860
Epoch 1387/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.6947 - disc_loss: 0.1708 - rmse: 0.1813 - val_loss: 0.3528
Epoch 1388/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8022 - disc_loss: 0.1612 - rmse: 0.2744 - val_loss: 0.2826
Epoch 1389/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.1474 - disc_loss: 0.1542 - rmse: 0.1701 - val_loss: 0.2695
Epoch 1390/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6851 - disc_loss: 0.1628 - rmse: 0.1928 - val_loss: 0.2421
Epoch 1391/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [38]:
print(real_df_all.shape)
rnn_target_column = 'tp'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(68664, 36)
tp


In [39]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (48064, 36) val_df.shape :  (13733, 36) test_df.shape: (6867, 36)


In [40]:
#fake_df.shape[0]/8760

In [41]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [42]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  6
out_num_features :  1


In [43]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [44]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/geum_auto_13/phosphorus/


In [47]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [48]:
rnn_epochs

20

## 모델 학습

In [49]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.3836 - mean_absolute_error: 0.4013 - nse: 0.0808
Epoch 00001: val_nse improved from -inf to 0.45686, saving model to save/best_model.h5
10/10 [==============================] - 10s 980ms/step - loss: 0.3836 - mean_absolute_error: 0.4013 - nse: 0.0808 - val_loss: 0.6259 - val_mean_absolute_error: 0.4883 - val_nse: 0.4569
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.2073 - mean_absolute_error: 0.2616 - nse: 0.5108
Epoch 00002: val_nse improved from 0.45686 to 0.71050, saving model to save/best_model.h5
10/10 [==============================] - 10s 955ms/step - loss: 0.2073 - mean_absolute_error: 0.2616 - nse: 0.5108 - val_loss: 0.3328 - val_mean_absolute_error: 0.3731 - val_nse: 0.7105
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.1439 - mean_absolute_error: 0.2191 - nse: 0.7413
Epoch 00003: val_nse improved from 0.71050 to 0.78043, saving model to save/best_model.h5
10/1

10/10 [==============================] - ETA: 0s - loss: 0.0639 - mean_absolute_error: 0.1504 - nse: 0.8456
Epoch 00022: val_nse did not improve from 0.81572
10/10 [==============================] - 9s 948ms/step - loss: 0.0639 - mean_absolute_error: 0.1504 - nse: 0.8456 - val_loss: 0.3172 - val_mean_absolute_error: 0.3553 - val_nse: 0.7188
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.0707 - mean_absolute_error: 0.1559 - nse: 0.8469
Epoch 00023: val_nse did not improve from 0.81572
10/10 [==============================] - 10s 955ms/step - loss: 0.0707 - mean_absolute_error: 0.1559 - nse: 0.8469 - val_loss: 0.3341 - val_mean_absolute_error: 0.3625 - val_nse: 0.7049
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.0476 - mean_absolute_error: 0.1437 - nse: 0.8665
Epoch 00024: val_nse did not improve from 0.81572
10/10 [==============================] - 10s 953ms/step - loss: 0.0476 - mean_absolute_error: 0.1437 - nse: 0.8665 - val_loss: 0.32

10/10 [==============================] - ETA: 0s - loss: 0.0503 - mean_absolute_error: 0.1348 - nse: 0.8683
Epoch 00045: val_nse did not improve from 0.81572
10/10 [==============================] - 10s 962ms/step - loss: 0.0503 - mean_absolute_error: 0.1348 - nse: 0.8683 - val_loss: 0.3280 - val_mean_absolute_error: 0.3408 - val_nse: 0.7073
Epoch 46/50
10/10 [==============================] - ETA: 0s - loss: 0.0460 - mean_absolute_error: 0.1355 - nse: 0.8888
Epoch 00046: val_nse did not improve from 0.81572
10/10 [==============================] - 10s 961ms/step - loss: 0.0460 - mean_absolute_error: 0.1355 - nse: 0.8888 - val_loss: 0.3804 - val_mean_absolute_error: 0.3652 - val_nse: 0.6630
Epoch 47/50
10/10 [==============================] - ETA: 0s - loss: 0.0487 - mean_absolute_error: 0.1371 - nse: 0.8777
Epoch 00047: val_nse did not improve from 0.81572
10/10 [==============================] - 10s 951ms/step - loss: 0.0487 - mean_absolute_error: 0.1371 - nse: 0.8777 - val_loss: 0.4

## core / window.py / 

In [50]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print('nse_val : ', val_nse['GRU'])
#print("---------------")
#print(val_nse['GRU'])
#print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.012575968
predict_day :  5
------------------------
0.0034214505 3.0477682e-05 0.0055206595
save/geum/models/geum_auto_13/phosphorus/
year : 2013 ~ 2020
run :  range(0, 1)
target :  tp   6
length :  68664
train=7, test=1, val=2
nse_val :  [0.74215]


In [51]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print('nse_all : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.011569086
predict_day :  5
------------------------
0.0023374301 1.5682988e-05 0.0039601754
nse_all :  [0.797455]


In [52]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.010682329
predict_day :  5
------------------------
0.0018809077 9.989271e-06 0.0031605808
nse_train :  [0.8152328]


In [53]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('nse_test : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.016334947
predict_day :  5
------------------------
0.003714183 2.5506248e-05 0.005050371
nse_test :  [0.833304]


In [32]:
#train_df.shape, val_df.shape, test_df.shape

In [33]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column, " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
